# Lag-Llama — Rolling Forecast (ctx=512, H in {1,3,5,20})


In [ ]:

# Install (only if needed)
# pip install "git+https://github.com/time-series-foundation-models/lag-llama.git"
# pip install pandas numpy torch matplotlib huggingface_hub

# source venv312-lagllama/bin/activate


In [ ]:

    import sys
    from pathlib import Path

    import numpy as np
    import pandas as pd
    import torch

    from utils.metrics import evaluate_forecast
    from utils.plotting import (
        plot_price_actual_vs_pred,
        plot_equity_no_tc_vs_bh,
        plot_equity_tc_vs_bh,
        plot_equity_no_tc_vs_bh_log,
        plot_equity_tc_vs_bh_log,
        plot_price_and_equity_panel,
    )
    import lagllama_no_gluonts as ll

    PROJECT_ROOT = Path("/Users/jan/Documents/working papers/project 1")
    if str(PROJECT_ROOT) not in sys.path:
        sys.path.insert(0, str(PROJECT_ROOT))

    DATA_CSV = PROJECT_ROOT / "data/btc_1h_test.csv"
    TIMESTAMP_COL = "timestamp"
    TARGET_COL = "close"

    ALLOWED_HORIZONS = (1, 3, 5, 20)
    HORIZON = 1  # choose 1, 3, 5, or 20
    if HORIZON not in ALLOWED_HORIZONS:
        raise ValueError(f"HORIZON must be one of {ALLOWED_HORIZONS}, got {HORIZON}")

    CONTEXT_LENGTH = 512
    FREQ = "1H"
    NUM_SAMPLES = 400
    ROPE_SCALE = 1.5

    OUT_PRED_CSV = PROJECT_ROOT / f"models/LagLlama_zeroshot_btc_preds_h{HORIZON}.csv"
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Device: {DEVICE}
Data: {DATA_CSV}")


In [ ]:

# data
df = pd.read_csv(DATA_CSV, usecols=[TIMESTAMP_COL, TARGET_COL]).rename(
    columns={TIMESTAMP_COL: "timestamp", TARGET_COL: "target"}
)
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=True)
df = df.dropna(subset=["timestamp", "target"]).sort_values("timestamp").reset_index(drop=True)

vals = df["target"].to_numpy(np.float32)
ts = df["timestamp"].to_numpy()

need = CONTEXT_LENGTH + HORIZON + 1
if len(vals) < need:
    raise ValueError(f"Not enough rows for CONTEXT+HORIZON: need {need}, have {len(vals)}")

idxs = np.arange(CONTEXT_LENGTH - 1, len(vals) - HORIZON)
print(f"Windows to forecast: {len(idxs)}")


In [ ]:

# load Lag-Llama core once
core, _ = ll.load_core(rope_scale=ROPE_SCALE)
print("Core device:", ll.DEVICE)


In [ ]:

# rolling forecast
preds = np.zeros((len(idxs), HORIZON), dtype=np.float32)

for j, i in enumerate(idxs):
    ctx_slice = slice(i - CONTEXT_LENGTH + 1, i + 1)
    ctx_series = pd.Series(vals[ctx_slice], index=pd.to_datetime(ts[ctx_slice]), dtype="float32")

    fc_df = ll.forecast_with_core(
        core,
        y=ctx_series,
        prediction_length=HORIZON,
        context_length=CONTEXT_LENGTH,
        freq=FREQ,
        num_samples=NUM_SAMPLES,
    )

    preds[j, :] = fc_df["mean"].to_numpy(np.float32)

    if j % 200 == 0:
        print(f"Processed {j}/{len(idxs)}")

base_ts = ts[idxs]
true_vals = [vals[i + 1 : i + 1 + HORIZON] for i in idxs]
future_ts = [ts[i + 1 : i + 1 + HORIZON] for i in idxs]

rows = []
for n in range(len(idxs)):
    for h in range(HORIZON):
        rows.append((base_ts[n], h + 1, future_ts[n][h], float(preds[n, h]), float(true_vals[n][h])))
df_long = pd.DataFrame(rows, columns=["base_timestamp", "horizon_step", "forecast_timestamp", "y_pred", "y_true"])

df_step1 = (
    df_long[df_long["horizon_step"] == 1][["forecast_timestamp", "y_true", "y_pred"]]
    .rename(columns={"forecast_timestamp": "timestamp"})
    .reset_index(drop=True)
)
df_step1.head()


In [ ]:

# save predictions
df_step1.to_csv(OUT_PRED_CSV, index=False)
print(f"Saved step1 predictions to: {OUT_PRED_CSV} (rows={len(df_step1)})")


In [ ]:

# evaluation
results = evaluate_forecast(
    y_true=df_step1["y_true"],
    y_pred=df_step1["y_pred"],
    timestamps=df_step1["timestamp"],
    starting_capital=100_000,
    threshold=0.005,
    fee_rate=0.001,
)

eq_end_no = float(results["equity_no_tc"].iloc[-1])
eq_end_tc = float(results["equity_tc"].iloc[-1])
cum_fee = float(results["cum_fee"].iloc[-1])

print(f"Final equity (no TC): {eq_end_no:,.2f}")
print(f"Final equity (with TC): {eq_end_tc:,.2f}")
print(f"Cum fees: {cum_fee:,.2f}")
print({k: results[k] for k in ("arc", "asd", "mdd", "ir_star", "ir_starstar")})

df_eval = df_step1.copy()
df_eval["equity"] = results["equity_no_tc"].to_numpy(float)
df_eval["equity_tc"] = results["equity_tc"].to_numpy(float)
df_eval["fee"] = results["fee"].to_numpy(float)
df_eval["cum_fee"] = results["cum_fee"]

df_eval.head(3)
df_eval.tail(3)


In [ ]:

# plots
plot_price_actual_vs_pred(df_eval, title=f"Lag-Llama rolling H={HORIZON} on BTC 1h")
plot_equity_no_tc_vs_bh(df_eval, title="Equity vs B&H")
plot_equity_tc_vs_bh(df_eval, title="Equity (with fees) vs B&H")
plot_equity_no_tc_vs_bh_log(df_eval, title="Equity vs B&H (log)")
plot_equity_tc_vs_bh_log(df_eval, title="Equity (with fees) vs B&H (log)")
plot_price_and_equity_panel(df_eval, title=f"Lag-Llama rolling H={HORIZON}")
